<a href="https://colab.research.google.com/github/omarcos0708/Predicao_Seguros/blob/main/predi%C3%A7%C3%A3o_seguros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Case: encotrar melhor modelo de MachineLearning para a predição do preço de seguro, com base no banco de dados.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('/content/insurance.xlsx')

In [3]:
df.head()

,idade,sexo,imc,quantidade_filhos,fumante,regiao,custos_seguro
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.560,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1341 entries, 0 to 1340
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   idade              1341 non-null   int64  
 1   sexo               1338 non-null   object 
 2   imc                1341 non-null   float64
 3   quantidade_filhos  1341 non-null   int64  
 4   fumante            1341 non-null   object 
 5   regiao             1341 non-null   object 
 6   custos_seguro      1341 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.5+ KB


In [5]:
df.dropna(inplace = True)

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
le = LabelEncoder()

le.fit(df.sexo)
df.sexo = le.transform(df.sexo)

le.fit(df.fumante)
df.fumante = le.transform(df.fumante)

le.fit(df.regiao)
df.regiao = le.transform(df.regiao)

In [8]:
df = pd.DataFrame(df)

In [9]:
df.head()

,idade,sexo,imc,quantidade_filhos,fumante,regiao,custos_seguro
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.560,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_norm = pd.DataFrame(scaler.fit_transform(df), index = df.index, columns = df.columns)
df_norm.head()

,idade,sexo,imc,quantidade_filhos,fumante,regiao,custos_seguro
0,0.021739,0.0,0.321227,0.0,1.0,1.000000,0.251611
1,0.000000,1.0,0.479150,0.2,0.0,0.666667,0.009636
2,0.217391,1.0,0.473500,0.6,0.0,0.666667,0.053115
3,0.326087,1.0,0.181464,0.0,0.0,0.333333,0.333010
4,0.304348,1.0,0.347592,0.0,0.0,0.333333,0.043816


In [11]:
import statsmodels.formula.api as smf
function = 'custos_seguro~idade+imc+fumante+quantidade_filhos+regiao'
model = smf.ols(formula = function,data = df_norm).fit()
print(model.summary())
#percebe-se que a variavel 'sexo' tem um valor P alto entao ela deve ser removida para melhor funcionamento do modelo

                            OLS Regression Results                            
Dep. Variable:          custos_seguro   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     802.2
Date:                Mon, 12 Dec 2022   Prob (F-statistic):               0.00
Time:                        21:03:10   Log-Likelihood:                 1230.3
No. Observations:                1338   AIC:                            -2449.
Df Residuals:                    1332   BIC:                            -2417.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0488      0.00

In [12]:
x = df_norm[['idade','imc','quantidade_filhos','fumante','regiao']]
y = df_norm[['custos_seguro']]

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state = 0)

lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [14]:
from sklearn import metrics
r_sq = lr.score(x,y)
print(r_sq)

0.7505629738411864


In [15]:
#erro medio absuluto
y_pred_train = lr.predict(x_train)
print("MAE:", metrics.mean_absolute_error(y_train, y_pred_train))

MAE: 0.06760649849946623


In [16]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(x_train, y_train)

<ipython-input-16-cd642ec9f9c8>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train, y_train)


RandomForestRegressor()

In [17]:
from sklearn import metrics
r_sq = rf.score(x,y)
print(r_sq)
y_pred_train = rf.predict(x_train)
print("MAE:", metrics.mean_absolute_error(y_train, y_pred_train))

y_pred_test = rf.predict(x_test)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred_test))

0.9526924729095997
MAE: 0.01676225329647704
MAE: 0.04310771474213458


In [18]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor()
ada.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor()

In [19]:
r_sq = ada.score(x,y)
print(r_sq)
y_pred_train = ada.predict(x_train)
print("MAE:", metrics.mean_absolute_error(y_train, y_pred_train))

y_pred_test = ada.predict(x_test)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred_test))

0.8315103543345708
MAE: 0.06272831418541643
MAE: 0.06117563456740712


In [23]:
from sklearn.ensemble import GradientBoostingRegressor
grb = GradientBoostingRegressor()
grb.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor()

In [24]:
r_sq = grb.score(x,y)
print(r_sq)
y_pred_train = grb.predict(x_train)
print("MAE:", metrics.mean_absolute_error(y_train, y_pred_train))

y_pred_test = grb.predict(x_test)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred_test))

0.8972805056179911
MAE: 0.03346236453828395
MAE: 0.03927401193485866


In [25]:
from sklearn.model_selection import GridSearchCV

parameters = { "max_depth": [5],
              "min_samples_leaf": [4],
              "min_samples_split": [2],
              "n_estimators": [200]}

grid_search = GridSearchCV(grb, parameters, scoring="r2", cv=2, n_jobs=-1)

In [27]:
grid_search.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'max_depth': [5], 'min_samples_leaf': [4],
                         'min_samples_split': [2], 'n_estimators': [200]},
             scoring='r2')

In [28]:
print(grid_search.best_estimator_)
print(grid_search.best_params_)

GradientBoostingRegressor(max_depth=5, min_samples_leaf=4, n_estimators=200)
{'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}


In [30]:
best_model = grid_search.best_estimator_

In [31]:
best_model.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [32]:
grb_tunned = GradientBoostingRegressor(alpha = 0.09,
 ccp_alpha = 0.0,
 criterion = 'friedman_mse',
 init = None,
 learning_rate = 0.1,
 loss = 'squared_error',
 max_depth = 5,
 max_features = None,
 max_leaf_nodes = None,
 min_impurity_decrease = 0.0,
 min_samples_leaf = 4,
 min_samples_split = 2,
 min_weight_fraction_leaf = 0.0,
 n_estimators = 200,
 n_iter_no_change = None,
 random_state = None,
 subsample = 1.0,
 tol = 0.0001,
 validation_fraction = 0.1,
 verbose = 0,
 warm_start = False)

In [33]:
grb_tunned.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.09, max_depth=5, min_samples_leaf=4,
                          n_estimators=200)

In [36]:
r_sq = grb_tunned.score(x,y)
r_sq

0.9325824302919626

In [34]:
y_pred_train = grb_tunned.predict(x_train)
print("MAE:", metrics.mean_absolute_error(y_train, y_pred_train))

MAE: 0.02383086606965511


In [35]:
y_pred_test = grb_tunned.predict(x_test)
print("MAE:", metrics.mean_absolute_error(y_test, y_pred_test))

MAE: 0.0457099737575093
